# 이지어드민

- 수집을 최근 30일로
- 저장은 나중에

In [23]:
import requests
import psycopg2
from datetime import datetime, timedelta
import time
import pandas as pd
import json
from psycopg2.extras import Json

In [2]:
def get_ez_order(start_date, end_date, page):


    partner_key = '73c6ac2e3366905793ab35d5a222d572'
    domain_key = '73c6ac2e33669057718abd505d76aa87366d3eed6643997457a0f9a03797aa9f'
   
    # limit = '100'
    

    url = f'http://api2.cloud.ezadmin.co.kr/ezadmin/function.php?partner_key={partner_key}&domain_key={domain_key}&action=get_order_info&date_type=collect_date&start_date={start_date}&end_date={end_date}&limit=1000&page={page}'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()['data']
    else:
        return ValueError

In [3]:
def save_ez_order(cursor, data):

    query = """
        INSERT INTO ez_order (seq, shop_id, shop_name, order_id, order_id_seq, shop_product_id, product_name, options, qty, order_name, order_mobile, order_tel, recv_name, recv_mobile, recv_tel, recv_address, recv_zip, memo, status, order_cs, collect_date, order_date, trans_date, trans_date_pos, supply_price, amount, extra_money, trans_corp, trans_no, trans_who, prepay_price, gift, hold, org_seq, deal_no, cust_id, order_products)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s::jsonb)
        ON CONFLICT (seq) DO UPDATE
        SET shop_id = EXCLUDED.shop_id, 
        shop_name = EXCLUDED.shop_name, 
        order_id = EXCLUDED.order_id, 
        order_id_seq = EXCLUDED.order_id_seq, 
        shop_product_id = EXCLUDED.shop_product_id, 
        product_name = EXCLUDED.product_name, 
        options = EXCLUDED.options, 
        qty = EXCLUDED.qty, 
        order_name = EXCLUDED.order_name,
        order_mobile = EXCLUDED.order_mobile,
        order_tel = EXCLUDED.order_tel,
        recv_name = EXCLUDED.recv_name,
        recv_mobile = EXCLUDED.recv_mobile,
        recv_tel = EXCLUDED.recv_tel,
        recv_address = EXCLUDED.recv_address,
        recv_zip = EXCLUDED.recv_zip,
        memo = EXCLUDED.memo,
        status = EXCLUDED.status,
        order_cs = EXCLUDED.order_cs,
        collect_date = EXCLUDED.collect_date,
        order_date = EXCLUDED.order_date,
        trans_date = EXCLUDED.trans_date,
        trans_date_pos = EXCLUDED.trans_date_pos,
        supply_price = EXCLUDED.supply_price,
        amount = EXCLUDED.amount,
        extra_money = EXCLUDED.extra_money,
        trans_corp = EXCLUDED.trans_corp,
        trans_no = EXCLUDED.trans_no,
        trans_who = EXCLUDED.trans_who,
        prepay_price = EXCLUDED.prepay_price,
        gift = EXCLUDED.gift,
        hold = EXCLUDED.hold,
        org_seq = EXCLUDED.org_seq,
        deal_no = EXCLUDED.deal_no,
        cust_id = EXCLUDED.cust_id,
        order_products = EXCLUDED.order_products
        ;
    """

    for d in data:
        cursor.execute(query, (d['seq'], d['shop_id'], d['shop_name'], d['order_id'],  d['order_id_seq'], d['shop_product_id'],  d['product_name'], d['options'], d['qty'], d['order_name'], d['order_mobile'], d['order_tel'], d['recv_name'], d['recv_mobile'], d['recv_tel'], d['recv_address'], d['recv_zip'], d['memo'], d['status'], d['order_cs'], d['collect_date'], d['order_date'], d['trans_date'], d['trans_date_pos'], d['supply_price'], d['amount'], d['extra_money'], d['trans_corp'], d['trans_no'], d['trans_who'], d['prepay_price'], d['gift'], d['hold'], d['org_seq'], d['deal_no'], d['cust_id'], json.dumps(d['order_products'])))



In [6]:
# start_date = datetime.strptime("2023-05-01", "%Y-%m-%d")
# end_date = datetime.strptime("2023-05-04", "%Y-%m-%d")
start_date = datetime.today() - timedelta(days=30)
end_date = datetime.today()

date_list = []
current_date = start_date

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    date_list.append(formatted_date)
    current_date += timedelta(days=1)

In [5]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()


for date in date_list:
    page = 1
    data = ['init']
    while len(data) > 0:
        data = get_ez_order(date, date, str(page))
        save_ez_order(cursor, data)
        print(len(data), date, page)
        page += 1
 

conn.commit()
conn.close()

0 2023-05-01 1
1000 2023-05-02 1
31 2023-05-02 2
0 2023-05-02 3
399 2023-05-03 1
0 2023-05-03 2
325 2023-05-04 1
0 2023-05-04 2


# 네이버

- 테스트로 테이블을 하나 만들어서 데이터 검증 필요
    - 2023 한달 단위로 비교
- 최근 30일로 만들기

In [172]:
import requests
import bcrypt
import pybase64
import time
import urllib.request
import urllib.parse
from datetime import datetime, timedelta
import pandas as pd
import psycopg2.extras


now = datetime.now()
date_list = []
for i in range(1, 31):
    date = now - timedelta(days=i)
    date_list.append(date.astimezone().isoformat())


def get_token(client_id, client_secret, type_="SELF") -> str:
    timestamp = str(int((time.time()-3) * 1000))
    pwd = f'{client_id}_{timestamp}'
    hashed = bcrypt.hashpw(pwd.encode('utf-8'), client_secret.encode('utf-8'))
    client_secret_sign = pybase64.standard_b64encode(hashed).decode('utf-8')

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data_ = {
        "client_id": client_id,
        "timestamp": timestamp,
        "client_secret_sign": client_secret_sign,
        "grant_type": "client_credentials",
        "type": type_
    }

    query = urllib.parse.urlencode(data_)
    url = 'https://api.commerce.naver.com/external/v1/oauth2/token?' + query
    res = requests.post(url=url, headers=headers)
    res_data = res.json()

    while True:
        if 'access_token' in res_data:
            token = res_data['access_token']
            return token
        else:
            print(f'[{res_data}] 토큰 요청 실패')
            time.sleep(1)


def get_naver_order(token, lastChangedFrom, moreSequence):
    
    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/last-changed-statuses'
   
    params = {
            'lastChangedFrom' : lastChangedFrom, #조회시작일시
            'moreSequence' : moreSequence,
        }

    res = requests.get(url=url, headers=headers, params=params)
    res_data = res.json()

    if 'data' not in res_data: #조회된 정보가 없을 경우 data키 없음
        print('주문 내역 없음')
        return False

    return res_data


def save_naver_order(cursor, data):
    query = """
        INSERT INTO naver_order ("lastChangedType", "orderId", "productOrderId", "productOrderStatus", "paymentDate", "lastChangedDate", "receiverAddressChanged")
        VALUES %s
        ON CONFLICT ("productOrderId") DO UPDATE
        SET "lastChangedType" = EXCLUDED."lastChangedType", 
        "orderId" = EXCLUDED."orderId", 
        "productOrderStatus" = EXCLUDED."productOrderStatus", 
        "paymentDate" = EXCLUDED."paymentDate", 
        "lastChangedDate" = EXCLUDED."lastChangedDate", 
        "receiverAddressChanged" = EXCLUDED."receiverAddressChanged";
    """

    rows = [(d['lastChangedType'], d['orderId'], d['productOrderId'], d['productOrderStatus'], d.get('paymentDate'), d['lastChangedDate'], d['receiverAddressChanged']) for d in data]

    # execute_values를 사용하여 여러 행 삽입
    psycopg2.extras.execute_values(cursor, query, rows)


def get_naver_order_detail(token, productOrderId):

    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/query'
    
    params = {
        'productOrderIds' : productOrderId
    }

    res = requests.post(url=url, headers=headers, json=params)
    res_data = res.json()
    
    if 'data' not in res_data:
        return False
    
    return res_data['data']


def save_naver_order_detail(cursor, data):

    columns = ['"cancelApprovalDate"', '"cancelCompletedDate"', '"cancelDetailedReason"', '"cancelReason"', '"claimRequestDate"', '"claimStatus"', '"refundExpectedDate"', '"refundStandbyReason"', '"refundStandbyStatus"', '"requestChannel"', '"deliveredDate"', '"deliveryCompany"', '"deliveryMethod"', '"deliveryStatus"', '"isWrongTrackingNumber"', '"pickupDate"', '"sendDate"', '"trackingNumber"', '"wrongTrackingNumberRegisteredDate"', '"wrongTrackingNumberType"', '"claimDeliveryFeeDemandAmount"', '"claimDeliveryFeePayMeans"', '"claimDeliveryFeePayMethod"', '"collectAddress"', '"collectCompletedDate"', '"collectDeliveryCompany"', '"collectDeliveryMethod"', '"collectStatus"', '"collectTrackingNumber"', '"etcFeeDemandAmount"', '"etcFeePayMeans"', '"etcFeePayMethod"', '"exchangeDetailedReason"', '"exchangeReason"', '"holdbackDetailedReason"', '"holdbackReason"', '"holdbackStatus"', '"reDeliveryMethod"', '"reDeliveryStatus"', '"reDeliveryCompany"', '"chargeAmountPaymentAmount"', '"checkoutAccumulationPaymentAmount"', '"generalPaymentAmount"', '"naverMileagePaymentAmount"', '"orderDate"', '"orderDiscountAmount"', '"orderId"', '"ordererId"', '"ordererName"', '"ordererTel"', '"paymentDate"', '"paymentDueDate"', '"paymentMeans"', '"isDeliveryMemoParticularInput"', '"payLocationType"', '"ordererNo"', '"payLaterPaymentAmount"', '"claimType"', '"decisionDate"', '"delayedDispatchDetailedReason"', '"delayedDispatchReason"', '"deliveryDiscountAmount"', '"deliveryFeeAmount"', '"deliveryPolicyType"', '"expectedDeliveryMethod"', '"freeGift"', '"mallId"', '"optionCode"', '"optionPrice"', '"packageNumber"', '"placeOrderDate"', '"placeOrderStatus"', '"productClass"', '"productDiscountAmount"', '"productId"', '"productName"', '"productOption"', '"productOrderId"', '"productOrderStatus"', '"quantity"', '"sectionDeliveryFee"', '"sellerProductCode"', '"shippingDueDate"', '"shippingFeeType"', '"shippingAddress"', '"shippingMemo"', '"totalPaymentAmount"', '"totalProductAmount"', '"unitPrice"', '"sellerBurdenDiscountAmount"', '"commissionRatingType"', '"commissionPrePayStatus"', '"paymentCommission"', '"saleCommission"', '"expectedSettlementAmount"', '"inflowPath"', '"itemNo"', '"optionManageCode"', '"sellerCustomCode1"', '"sellerCustomCode2"', '"claimId"', '"channelCommission"', '"individualCustomUniqueCode"', '"productImediateDiscountAmount"', '"productProductDiscountAmount"', '"productMultiplePurchaseDiscountAmount"', '"sellerBurdenImediateDiscountAmount"', '"sellerBurdenProductDiscountAmount"', '"sellerBurdenMultiplePurchaseDiscountAmount"', '"knowledgeShoppingSellingInterlockCommission"', '"giftReceivingStatus"', '"sellerBurdenStoreDiscountAmount"', '"sellerBurdenMultiplePurchaseDiscountType"', '"logisticsCompanyId"', '"logisticsCenterId"', '"arrivalGuaranteeDate"', '"deliveryAttributeType"', '"returnDetailedReason"', '"returnReason"', '"returnReceiveAddress"', '"returnCompletedDate"', '"holdbackConfigDate"', '"holdbackConfigurer"', '"holdbackReleaseDate"', '"holdbackReleaser"', '"claimDeliveryFeeProductOrderIds"', '"claimDeliveryFeeDiscountAmount"', '"remoteAreaCostChargeAmount"']
    rows = [(d.get('cancelApprovalDate'), d.get('cancelCompletedDate'), d.get('cancelDetailedReason'), d.get('cancelReason'), d.get('claimRequestDate'), d.get('claimStatus'), d.get('refundExpectedDate'), d.get('refundStandbyReason'), d.get('refundStandbyStatus'), d.get('requestChannel'), d.get('deliveredDate'), d.get('deliveryCompany'), d.get('deliveryMethod'), d.get('deliveryStatus'), d.get('isWrongTrackingNumber'), d.get('pickupDate'), d.get('sendDate'), d.get('trackingNumber'), d.get('wrongTrackingNumberRegisteredDate'), d.get('wrongTrackingNumberType'), d.get('claimDeliveryFeeDemandAmount'), d.get('claimDeliveryFeePayMeans'), d.get('claimDeliveryFeePayMethod'), json.dumps(d.get('collectAddress')), d.get('collectCompletedDate'), d.get('collectDeliveryCompany'), d.get('collectDeliveryMethod'), d.get('collectStatus'), d.get('collectTrackingNumber'), d.get('etcFeeDemandAmount'), d.get('etcFeePayMeans'), d.get('etcFeePayMethod'), d.get('exchangeDetailedReason'), d.get('exchangeReason'), d.get('holdbackDetailedReason'), d.get('holdbackReason'), d.get('holdbackStatus'), d.get('reDeliveryMethod'), d.get('reDeliveryStatus'), d.get('reDeliveryCompany'), d.get('chargeAmountPaymentAmount'), d.get('checkoutAccumulationPaymentAmount'), d.get('generalPaymentAmount'), d.get('naverMileagePaymentAmount'), d.get('orderDate'), d.get('orderDiscountAmount'), d['orderId'], d.get('ordererId'), d.get('ordererName'), d.get('ordererTel'), d.get('paymentDate'), d.get('paymentDueDate'), d.get('paymentMeans'), d.get('isDeliveryMemoParticularInput'), d.get('payLocationType'), d.get('ordererNo'), d.get('payLaterPaymentAmount'), d.get('claimType'), d.get('decisionDate'), d.get('delayedDispatchDetailedReason'), d.get('delayedDispatchReason'), d.get('deliveryDiscountAmount'), d.get('deliveryFeeAmount'), d.get('deliveryPolicyType'), d.get('expectedDeliveryMethod'), d.get('freeGift'), d.get('mallId'), d.get('optionCode'), d.get('optionPrice'), d.get('packageNumber'), d.get('placeOrderDate'), d.get('placeOrderStatus'), d.get('productClass'), d.get('productDiscountAmount'), d.get('productId'), d.get('productName'), d.get('productOption'), d['productOrderId'], d.get('productOrderStatus'), d.get('quantity'), d.get('sectionDeliveryFee'), d.get('sellerProductCode'), d.get('shippingDueDate'), d.get('shippingFeeType'), json.dumps(d.get('shippingAddress')), d.get('shippingMemo'), d.get('totalPaymentAmount'), d.get('totalProductAmount'), d.get('unitPrice'), d.get('sellerBurdenDiscountAmount'), d.get('commissionRatingType'), d.get('commissionPrePayStatus'), d.get('paymentCommission'), d.get('saleCommission'), d.get('expectedSettlementAmount'), d.get('inflowPath'), d.get('itemNo'), d.get('optionManageCode'), d.get('sellerCustomCode1'), d.get('sellerCustomCode2'), d.get('claimId'), d.get('channelCommission'), d.get('individualCustomUniqueCode'), d.get('productImediateDiscountAmount'), d.get('productProductDiscountAmount'), d.get('productMultiplePurchaseDiscountAmount'), d.get('sellerBurdenImediateDiscountAmount'), d.get('sellerBurdenProductDiscountAmount'), d.get('sellerBurdenMultiplePurchaseDiscountAmount'), d.get('knowledgeShoppingSellingInterlockCommission'), d.get('giftReceivingStatus'), d.get('sellerBurdenStoreDiscountAmount'), d.get('sellerBurdenMultiplePurchaseDiscountType'), d.get('logisticsCompanyId'), d.get('logisticsCenterId'), d.get('arrivalGuaranteeDate'), d.get('deliveryAttributeType'), d.get('returnDetailedReason'), d.get('returnReason'), json.dumps(d.get('returnReceiveAddress')), d.get('returnCompletedDate'), d.get('holdbackConfigDate'), d.get('holdbackConfigurer'), d.get('holdbackReleaseDate'), d.get('holdbackReleaser'), d.get('claimDeliveryFeeProductOrderIds'), d.get('claimDeliveryFeeDiscountAmount'), d.get('remoteAreaCostChargeAmount')) for d in data]
    
    query = """
    INSERT INTO naver_order_product2 ({}) VALUES %s
    ON CONFLICT ("productOrderId") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)

In [173]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
# print(f'발급된 토큰 : ', token)


data_list = []


for date in date_list:
    data = get_naver_order(token, date, None)
    data_list += data['data']['lastChangeStatuses']
    print(data['data']['count'])
    time.sleep(3)
    

    while 'more' in data['data'].keys():
        moreFrom = data['data']['more']['moreFrom']
        moreSequence = data['data']['more']['moreSequence']
        data = get_naver_order(token, moreFrom, moreSequence)
        data_list += data['data']['lastChangeStatuses']
        print(data['data']['count'])
        time.sleep(3) # 나중엔 5초로 변경


save_naver_order(cursor, data_list)


conn.commit()
conn.close()

300
213
252
207
213
240
278
277
247
208
135
191
139
163
192
173
107
121
180
215
179
111
177
123
163
138
213
195
237
96
113


In [174]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
productOrderId = [d['productOrderId'] for d in data_list]
data_list_2 = []


for i in range(0, len(productOrderId), 300):
    data = get_naver_order_detail(token, productOrderId[i:i+300])

    for d in data:
        data_dict = {}
        keys = d.keys()
        for key in keys:
            data_dict.update(d[key])
        data_list_2.append(data_dict)
    

save_naver_order_detail(cursor, data_list_2)


conn.commit()
conn.close()

# 쿠팡

In [187]:
import os
import time
import hmac, hashlib
import urllib.parse
import urllib.request
import ssl
import json
import pandas as pd
from datetime import datetime, timedelta


def get_coupang_order(createdAtFrom, createdAtTo, status):
    

    os.environ['TZ'] = 'GMT+0'
    date=(datetime.now() - timedelta(hours = 9)).strftime('%y%m%dT%H%M%SZ')
    method = "GET"
    print(date)
    path = f"/v2/providers/openapi/apis/api/v4/vendors/A00197911/ordersheets"
    query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "status": status})
    message = date+method+path+query


    accesskey = "73814036-d7f4-4d56-a254-35908249a0c6"
    secretkey = "86ecbb8dcd4f4d9c4bd83f724ed4bfb4919ba0cc"

    #********************************************************#
    #authorize, demonstrate how to generate hmac signature here
    signature=hmac.new(secretkey.encode('utf-8'),message.encode('utf-8'),hashlib.sha256).hexdigest()
    authorization  = "CEA algorithm=HmacSHA256, access-key="+accesskey+", signed-date="+date+", signature="+signature
    #print out the hmac key
    #print(authorization)
    #********************************************************#

    # ************* SEND THE REQUEST *************
    url = "https://api-gateway.coupang.com"+path+"?%s" % query

    req = urllib.request.Request(url)

    req.add_header("Content-type","application/json;charset=UTF-8")
    req.add_header("Authorization",authorization)

    req.get_method = lambda: method

    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    try:
        resp = urllib.request.urlopen(req,context=ctx)
    except urllib.request.HTTPError as e:
        print(e.code)
        print(e.reason)
        print(e.fp.read())
    except urllib.request.URLError as e:
        print(e.errno)
        print(e.reason)
        print(e.fp.read())
    else:
        # 200
        body = resp.read().decode(resp.headers.get_content_charset())
        return body
   

In [178]:
status_list = ['ACCEPT', 'INSTRUCT', 'DEPARTURE', 'DELIVERING', 'FINAL_DELIVERY']

In [192]:
test = get_coupang_order('2023-06-23', '2023-06-23', 'FINAL_DELIVERY')

230704T090202Z


In [196]:
test = json.loads(test)

In [199]:
len(test['data'])

50

In [191]:
# 시작일 = datetime.datetime.today().strftime("%Y-%m-%d") # 오늘날짜
# 종료일 = datetime.datetime.today().strftime("%Y-%m-%d") # 오늘날짜
# 2023-06-23 FINAL_DELIVERY 5039624910

for date in date_list:
    for status in status_list:
        test = get_coupang_order(date, date, status)
        tt = json.loads(test)   
        print(date, status, tt['nextToken'])

230704T085944Z
2023-07-03 ACCEPT 
230704T085944Z
2023-07-03 INSTRUCT 
230704T085944Z
2023-07-03 DEPARTURE 
230704T085944Z
2023-07-03 DELIVERING 
230704T085945Z
2023-07-03 FINAL_DELIVERY 
230704T085945Z
2023-07-02 ACCEPT 
230704T085945Z
2023-07-02 INSTRUCT 
230704T085945Z
2023-07-02 DEPARTURE 
230704T085946Z
2023-07-02 DELIVERING 
230704T085946Z
2023-07-02 FINAL_DELIVERY 
230704T085947Z
2023-07-01 ACCEPT 
230704T085947Z
2023-07-01 INSTRUCT 
230704T085947Z
2023-07-01 DEPARTURE 
230704T085947Z
2023-07-01 DELIVERING 
230704T085948Z
2023-07-01 FINAL_DELIVERY 
230704T085948Z
2023-06-30 ACCEPT 
230704T085948Z
2023-06-30 INSTRUCT 
230704T085949Z
2023-06-30 DEPARTURE 
230704T085949Z
2023-06-30 DELIVERING 
230704T085949Z
2023-06-30 FINAL_DELIVERY 
230704T085949Z
2023-06-29 ACCEPT 
230704T085950Z
2023-06-29 INSTRUCT 
230704T085950Z
2023-06-29 DEPARTURE 
230704T085950Z
2023-06-29 DELIVERING 
230704T085950Z
2023-06-29 FINAL_DELIVERY 
230704T085951Z
2023-06-28 ACCEPT 
230704T085951Z
2023-06-28 INSTR

KeyboardInterrupt: 

In [188]:
now = datetime.now()
date_list = []
for i in range(1, 100):
    date = now - timedelta(days=i)
    date_list.append(date.strftime("%Y-%m-%d"))

In [189]:
date_list

['2023-07-03',
 '2023-07-02',
 '2023-07-01',
 '2023-06-30',
 '2023-06-29',
 '2023-06-28',
 '2023-06-27',
 '2023-06-26',
 '2023-06-25',
 '2023-06-24',
 '2023-06-23',
 '2023-06-22',
 '2023-06-21',
 '2023-06-20',
 '2023-06-19',
 '2023-06-18',
 '2023-06-17',
 '2023-06-16',
 '2023-06-15',
 '2023-06-14',
 '2023-06-13',
 '2023-06-12',
 '2023-06-11',
 '2023-06-10',
 '2023-06-09',
 '2023-06-08',
 '2023-06-07',
 '2023-06-06',
 '2023-06-05',
 '2023-06-04',
 '2023-06-03',
 '2023-06-02',
 '2023-06-01',
 '2023-05-31',
 '2023-05-30',
 '2023-05-29',
 '2023-05-28',
 '2023-05-27',
 '2023-05-26',
 '2023-05-25',
 '2023-05-24',
 '2023-05-23',
 '2023-05-22',
 '2023-05-21',
 '2023-05-20',
 '2023-05-19',
 '2023-05-18',
 '2023-05-17',
 '2023-05-16',
 '2023-05-15',
 '2023-05-14',
 '2023-05-13',
 '2023-05-12',
 '2023-05-11',
 '2023-05-10',
 '2023-05-09',
 '2023-05-08',
 '2023-05-07',
 '2023-05-06',
 '2023-05-05',
 '2023-05-04',
 '2023-05-03',
 '2023-05-02',
 '2023-05-01',
 '2023-04-30',
 '2023-04-29',
 '2023-04-

In [ ]:
# status
# next_token
# return_cancel
# save
# 최근 30일